# Module import

In [1]:
from ETGEMs_function import *

# Data initialization

In [2]:
import pandas as pd
import cobra
#Get Reaction G0 from local file
reaction_g0_file='./Basic Data/reaction_g0.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file = './Basic Data/metabolites_lnC.txt'
#Get Model from local file
model_file='./Basic Data/iML1515.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file='./Basic Data/ID_kcat_MW_file.csv'

Concretemodel_Need_Data=Get_Concretemodel_Need_Data(reaction_g0_file,metabolites_lnC_file,model_file,reaction_kcat_MW_file)

# Calculation of thermodynamic driving force of reactions
Without metabolite ratio constraints;
K value eaual to (maxDFi-minDFi).

In [3]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_Df_list=pd.DataFrame()

#Maximize (DFi), without metabolites ratio constraint
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df,Concretemodel_Need_Data,eachreaction,'maximize','gurobi'): eachreaction for eachreaction in list(Concretemodel_Need_Data['reaction_g0'].index)}
    for future in as_completed(futures):
        tmp=future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list.loc[eachindex,'max_value']=tmp.loc[eachindex,'max_value']
#Minimize (DFi), without metabolites ratio constraint
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df,Concretemodel_Need_Data,eachreaction,'minimize','gurobi'): eachreaction for eachreaction in list(Concretemodel_Need_Data['reaction_g0'].index)}
    for future in as_completed(futures):
        tmp=future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list.loc[eachindex,'min_value']=tmp.loc[eachindex,'min_value']

max_min_Df_list['diff']=max_min_Df_list['max_value']-max_min_Df_list['min_value']
endtime = datetime.datetime.now()
print (endtime - starttime)
max_min_Df_list.to_csv('./Analysis Result/max_min_Df_list_no_fixed_ratio.csv', sep=',', header=True, index=True,mode='w')
print("Max Df : "+str(np.max(max_min_Df_list['max_value'])))
print("Min Df : "+str(np.min(max_min_Df_list['min_value'])))
print("K : "+str(np.max(max_min_Df_list['max_value'])-np.min(max_min_Df_list['min_value'])))

0:22:44.249949
Max Df : 828.958488834085
Min Df : -419.46287974197605
K : 1248.421368576061


# Calculation of thermodynamic driving force for reactions¶
 Under constraints of several fixed metabolite ratio.

In [4]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_Df_list_fixed=pd.DataFrame()

#Maximize (DFi), under metabolites ratio constraints
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df_Ratio,Concretemodel_Need_Data,eachreaction,'maximize','gurobi'): eachreaction for eachreaction in list(Concretemodel_Need_Data['reaction_g0'].index)}
    for future in as_completed(futures):
        tmp=future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list_fixed.loc[eachindex,'max_Df under fixed Ci ratio']=tmp.loc[eachindex,'max_value']
#Minimize (DFi), under metabolites ratio constraints
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df_Ratio,Concretemodel_Need_Data,eachreaction,'minimize','gurobi'): eachreaction for eachreaction in list(Concretemodel_Need_Data['reaction_g0'].index)}
    for future in as_completed(futures):
        tmp=future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list_fixed.loc[eachindex,'min_Df under fixed Ci ratio']=tmp.loc[eachindex,'min_value']

max_min_Df_list_fixed['diff under fixed Ci ratio']=max_min_Df_list_fixed['max_Df under fixed Ci ratio']-max_min_Df_list_fixed['min_Df under fixed Ci ratio']
endtime = datetime.datetime.now()
print (endtime - starttime)
max_min_Df_list_fixed.to_csv('./Analysis Result/max_min_Df_list_fixed_ratio.csv', sep=',', header=True, index=True,mode='w')
print("Max Df : "+str(np.max(max_min_Df_list_fixed['max_Df under fixed Ci ratio'])))
print("Min Df : "+str(np.min(max_min_Df_list_fixed['min_Df under fixed Ci ratio'])))
print("K : "+str(np.max(max_min_Df_list_fixed['max_Df under fixed Ci ratio'])-np.min(max_min_Df_list_fixed['min_Df under fixed Ci ratio'])))

0:22:43.632193
Max Df : 828.958488834085
Min Df : -419.46287974197605
K : 1248.421368576061


# Comparison of two groups data
The results are shown in Table D in Supplementary file2.

In [5]:
max_min_Df_total_record=pd.concat([max_min_Df_list,max_min_Df_list_fixed],axis=1)
max_min_Df_total_record=max_min_Df_total_record.sort_values(by='max_Df under fixed Ci ratio',ascending = True)
max_min_Df_total_record=max_min_Df_total_record[max_min_Df_total_record['max_Df under fixed Ci ratio'] < 0]
max_min_Df_total_record#.head(30)

,max_value,min_value,diff,max_Df under fixed Ci ratio,min_Df under fixed Ci ratio,diff under fixed Ci ratio
INOSTO_reverse,-349.353451,-419.462880,70.109429,-349.353451,-419.462880,70.109429
SHK3Dr_reverse,-22.642558,-131.957442,109.314884,-55.909646,-110.567088,54.657442
PCNO,-4.942558,-114.257442,109.314884,-38.209646,-92.867088,54.657442
E4PD_reverse,7.357442,-101.957442,109.314884,-25.909646,-80.567088,54.657442
ALDD19xr_reverse,13.557442,-95.757442,109.314884,-19.709646,-74.367088,54.657442
DXYLTD_reverse,-14.571279,-69.228721,54.657442,-14.571279,-69.228721,54.657442
RHMND_reverse,-14.571279,-69.228721,54.657442,-14.571279,-69.228721,54.657442
CBMKr,11.903974,-115.226011,127.129985,-9.486380,-81.958923,72.472543
GGGABADr_reverse,12.957442,-96.357442,109.314884,-8.432912,-63.090354,54.657442
DHBD_reverse,25.257442,-84.057442,109.314884,-8.009646,-62.667088,54.657442
